01 | `requests`
====

In [47]:
from pprint import pprint #El módulo pprint contiene una «impresora bonita» para producir vistas estéticamente agradables de estructuras de datos.

import requests # pedido 
from bs4 import BeautifulSoup

El uso más común que un científico de datos le va a dar a `requests` es interactuar con APIs para descargar datos. Esto implica usar la función `get` del paquete, que como dice su nombre, envía un GET a una URL que le indiquemos.

Una API es un intermediario entre dos sistemas, que permite que una aplicación se comunique con otra y pida datos o acciones específicas.

El siguiente bloque de código consulta una URL de prueba para que devuelva todos los posts disponibles (imaginar que es un foro en donde varios usuarios pueden crear artículo). Luego consultamos el `status_code` del request para confirmar que la URL funciona ([ver status posibles](https://developer.mozilla.org/es/docs/Web/HTTP/Status)) y decodificamos el contenido a formato JSON.

https://jsonplaceholder.typicode.com/posts

In [48]:
response = requests.get("https://jsonplaceholder.typicode.com/posts")
type(response)


requests.models.Response

In [49]:
print(response.status_code, "\n") # "\n" salto de línea,  si todo ha ido bien, deberíamos obtener un código 200
print(response.headers, "\n") 
pprint(response.json()[:5])

200 

{'Date': 'Wed, 22 May 2024 21:21:59 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Report-To': '{"group":"heroku-nel","max_age":3600,"endpoints":[{"url":"https://nel.heroku.com/reports?ts=1708469586&sid=e11707d5-02a7-43ef-b45e-2cf4d2036f7d&s=BRy1cTb48aKJuE5epAHdlAmiBGy0htic7FTaUAJgCdY%3D"}]}', 'Reporting-Endpoints': 'heroku-nel=https://nel.heroku.com/reports?ts=1708469586&sid=e11707d5-02a7-43ef-b45e-2cf4d2036f7d&s=BRy1cTb48aKJuE5epAHdlAmiBGy0htic7FTaUAJgCdY%3D', 'Nel': '{"report_to":"heroku-nel","max_age":3600,"success_fraction":0.005,"failure_fraction":0.05,"response_headers":["Via"]}', 'X-Powered-By': 'Express', 'X-Ratelimit-Limit': '1000', 'X-Ratelimit-Remaining': '999', 'X-Ratelimit-Reset': '1708469640', 'Vary': 'Origin, Accept-Encoding', 'Access-Control-Allow-Credentials': 'true', 'Cache-Control': 'max-age=43200', 'Pragma': 'no-cache', 'Expires': '-1', 'X-Content-Type-Options': 'nosniff', 'Etag': 'W/"6b80

En el bloque anterior se consultó el JSON directamente porque esa web devuelve resultados en JSON. Si no fuera así, en lugar de llamar al método `json` habría que ver los atributos `content` (si el contenido fuera binario) o `text` (en el resto de los casos). Veamos un caso de `text` con un informe epidemiológico del SINAE, para el cual usamos brevemente `BeautifulSoup` solo para hacer print del HTML prolijamente.

In [50]:
r2 = response.json()[:5]

In [51]:
type(r2)

list

In [52]:
r = requests.get("https://www.presidencia.gub.uy/comunicacion/comunicacionnoticias/informe-sinae-19-abril-2021")
print(r.status_code)
soup = BeautifulSoup(r.text)
print(soup.prettify())

200
<!DOCTYPE html>
<html dir="ltr" lang="es" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# schema: http://schema.org/ sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# ">
 <head>
  <meta charset="utf-8"/>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-135104282-1">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];function gtag(){dataLayer.push(arguments)};gtag("js", new Date());gtag("set", "developer_id.dMDhkMT", true);gtag("config", "UA-135104282-1", {"groups":"default","anonymize_ip":true,"page_placeholder":"PLACEHOLDER_page_path","allow_ad_personalization_signals":false});gtag("config", "G-RDFBN794BE", {"groups":"default","page_placeholder":"PLACEHOLDER_page_location","allow_ad_personalization_signals":fa

El método prettify() convertirá un análisis de Beautiful Soup en una cadena Unicode con un formato agradable, con una línea separada para cada etiqueta y cada cadena.

`requests` puede emular todos los protocolos de HTTP (POST, PUT, DELETE, HEAD, etc.). El siguiente bloque crea un nuevo artículo mediante la función `post`.

#### Parámetros POST
Una petición POST, por lo general, siempre va acompañada de una serie de parámetros que típicamente podemos encontrar en un formulario web. Es posible realizar estas peticiones en requests adjuntando los parámetros que necesitemos en el mismo formato de diccionario

Dado que se está creando un objeto, es necesario pasar algún tipo de data que debe estar en el formato que el sitio web espera.

In [53]:
r = requests.post("https://jsonplaceholder.typicode.com/posts", data={"title": "foo", "body": "bar", "userId": 1})
pprint(r.json()) #trae asociado un anexo de información, le pido info a la pag web y le paso datos

{'body': 'bar', 'id': 101, 'title': 'foo', 'userId': '1'}


Piensa en POST como una forma de "enviar" información a un servidor web. Cuando usas POST, estás entregando datos al servidor para que los procese de alguna manera. Aquí tienes un ejemplo simple de cuándo podrías usar POST:

Enviar un formulario: Imagina que estás completando un formulario en línea para registrarte en un sitio web. Cuando haces clic en "Enviar", los datos que ingresaste (como tu nombre, dirección de correo electrónico, contraseña, etc.) se envían al servidor utilizando POST

###### Podemos construir la URL de consulta directamente en `get`, lo que abre la puerta a oportunidades de automatización.

In [54]:
payload = {"c": "213,223,298,", "s": "NGDPDPC,", "sy": "2000", "ey": "2020", "ssm": "0", "scsm": "1", "scc": "0", "ssd": "1", "ssc": "0", "sic": "0", "sort": "country", "ds": ".", "br": "1"}
# payload(datos)
# "c": "213,223,298 paises, "sy" año 
r = requests.get("https://www.imf.org/en/Publications/WEO/weo-database/2020/October/weo-report?", params=payload)
r.url

'https://www.imf.org/en/Publications/WEO/weo-database/2020/October/weo-report?c=213%2C223%2C298%2C&s=NGDPDPC%2C&sy=2000&ey=2020&ssm=0&scsm=1&scc=0&ssd=1&ssc=0&sic=0&sort=country&ds=.&br=1'

https://www.imf.org/en/Publications/WEO/weo-database/2020/October/select-country-group

https://www.imf.org/en/Publications/WEO/weo-database/2020/October/select-subjects?c=213 (esta aca es ARG)

In [55]:
# &s=NGDP_R,&sy=2024&ey=2025&ssm  el dato que quise Gross domestic product, constant prices año 2024 y 2025

##### para que ??  los parametros variables me permiten buscar diferentes datos

In [56]:
import pandas as pd

In [57]:
pd.read_html(r.text)[0]

,Country,Subject Descriptor,Units,Scale,Country/Series-specific Notes,2000,2001,2002,2003,2004,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Argentina,"Gross domestic product per capita, current prices",U.S. dollars,Units,NaN,8638.543,8085.356,2997.621,3761.088,4314.098,...,12787.806,13889.792,14488.829,13208.831,14895.316,12772.868,14618.327,11624.891,9890.314,8433.039
1,Brazil,"Gross domestic product per capita, current prices",U.S. dollars,Units,NaN,3772.050,3177.940,2856.026,3089.141,3660.020,...,13295.899,12424.946,12358.342,12175.681,8846.478,8757.304,9974.814,9043.237,8751.381,6450.451
2,Uruguay,"Gross domestic product per capita, current prices",U.S. dollars,Units,NaN,6817.363,6238.214,4073.418,3614.627,4102.584,...,14054.330,14961.681,16723.447,16572.362,15365.956,15139.120,17041.722,17064.595,16110.742,15331.717


Podríamos consultar varias URLs secuencialmente y guardar las respuestas. En ese caso es buena práctica usar un objeto `Session`, que reutiliza la conexión y reduce el tiempo de ejecución. Además, la información que necesitemos proveerle al sitio web puede ser establecida en la sesión una sola vez, en lugar de en cada request (por ejemplo, usuario y contraseña).

In [58]:
s = requests.Session()
urls = ["https://www.cartelera.com.uy/averespectaculo.aspx?3305", 
        "https://www.cartelera.com.uy/averespectaculo.aspx?13766",
        "https://www.cartelera.com.uy/averespectaculo.aspx?3129"]
responses = []
for url in urls:
    r = s.get(url)
    responses.append(r.text)

responses[1]

'<!DOCTYPE html>\n<html lang="es" class="showTabs color-cine">\n\t<head>\n\t\t<meta charset=utf-8 />\n\t\t<title>En primera plana - Cartelera</title>\n\n\t\t\n<meta name="description" content=" Cuando el tenaz equipo de reporteros de The Boston Globe ahonda en los alegatos de abuso a menores dentro de la Iglesia Católica, descubre en su investigación el encubrimiento llevado a cabo durante décadas por parte de las altas esferas de organizaciones religiosas, legales y gubernamentales de Boston, desatando una ola de revelaciones alrededor del mundo. Basada en la historia real de la investigación ganadora del Premio Pulitzer. Oscar a la mejor película de 2015. " />\n<meta name="keywords" content="En primera plana,  Mark Ruffalo -Michael Keaton,  Tom McCarthy,  Estados Unidos " />\n\n\n\t\t<link rel="apple-touch-icon" sizes="180x180" href="https://cartelera.montevideo.com.uy/images17/favicon/apple-touch-icon.png">\n\t\t<link rel="icon" type="image/png" sizes="32x32" href="https://cartelera

La respuesta del servidor puede ser un archivo, y usamos `requests` para descargarlo. Por ejemplo, descarguemos el PDF de proyectos de inversión promovidos por el gobierno en 2020 desde la página del Ministerio de Economía.

In [59]:
url_2020 = "https://www.gub.uy/ministerio-economia-finanzas/sites/ministerio-economia-finanzas/files/2021-01/comap_setiembre-2020.pdf"
r_2020 = requests.get(url_2020)
r_2020.status_code

200

Recordar que `get()` devuelve un `Response` object. En este caso, este objeto es un PDF. Para descargarlo vamos a "escribir" su contenido en un archivo usando un context manager. Tenemos que elegir el modo "wb" ya que vamos a escribir (**w**rite) un archivo **b**inario.

En Python, el modo "wb" utilizado en la función open() indica que el archivo se abrirá en modo de escritura binaria. Aquí está lo que hace cada letra:

'w': Indica que el archivo se abrirá en modo de escritura (write mode). Esto significa que si el archivo ya existe, se truncará (es decir, se eliminará su contenido anterior). Si el archivo no existe, se creará uno nuevo.

'b': Indica que el archivo se abrirá en modo binario (binary mode). Esto es importante cuando trabajas con archivos que contienen datos binarios, como imágenes, archivos PDF, archivos de audio, etc. El modo binario asegura que los datos se lean o 

In [60]:
with open("2020.pdf", "wb") as f:
    f.write(r_2020.content)